# Generating LLM Responses to Customer Service Requests

This notebook demonstrates how to instruct a LLM to simulate the role of a customer service agent and generate responses to customer service queries.

See how you can evaluate the quality of these LLM generated responses in this tutorial: [Detecting Issues in LLM Outputs](https://help.cleanlab.ai/tutorials/llm/)

## Import Dependencies

Make sure you have wget installed to run this tutorial. You can use pip to install all other packages required for this tutorial as follows:

In [1]:
!pip install pandas scipy openai transformers torch accelerate bitsandbytes

In [2]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import openai

from tqdm import tqdm

## Fetch and View Customer Service Requests

To fetch the data for this tutorial, make sure you have wget installed.

In [ ]:
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/llm-customer-service-prompts.csv -P data

For this notebook we have a csv file that contains various customer service queries, let's view a selection of these requests:

In [4]:
data = pd.read_csv("data/llm-customer-service-prompts.csv")

requests = data["request"].values
requests[:5]

array(['help trying to update the delivery address',
       'want help to get my invoices from {{Person Name}}',
       'find information about the termination of my {{Account Type}} account',
       'could you help me to correct my shipping address?',
       'i cannot make payments help to report an error'], dtype=object)

## Obtaining Responses from Llama 2 

First, we demonstrate how to generate responses to our customer service requests using Llama 2, we can load the model from Hugging Face and create a pipeline for text generation.

Note that this portion of the notebook requires GPU, if that is unavailable, scroll to the next section to see how to obtain responses using OpenAI's API.

In [ ]:
model_id = 'meta-llama/Llama-2-7b-chat-hf' 
# model_id = "mistralai/Mistral-7B-v0.1"  # alternatively, can use mistral by changing the model_id
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto", 
)

In [6]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=400,
    temperature=0.8, # can adjust temp 
)

Next, we create a prompt template which we will use to format the inputs to the LLM.

In [7]:
def format_llama_prompt(text):
    return f"""You are a customer service agent, provide a response with next steps to the following question: 
    {text}
    
    Answer:"""

Finally, we pass the requests to the LLM using the formatted prompt and save its outputs.

In [ ]:
text_gen = (format_llama_prompt(t) for t in requests)
llama2_outputs = np.zeros(len(requests)).astype(object)

for i, out in tqdm(enumerate(pipe(text_gen))):
    llama2_outputs[i] = out[0]["generated_text"].split("Answer:")[1].strip()

Let's view a sample response from Llama 2:

In [9]:
print(llama2_outputs[0])

Thank you for reaching out to us. I apologize for any inconvenience you're experiencing with your delivery address. To update your delivery address, please follow these steps:

1. Log in to your account on our website or mobile app.
2. Click on the "Account" or "Settings" tab.
3. Scroll down to the "Delivery" section and click on "Edit."
4. Enter your new delivery address and confirm.
5. If you have any trouble updating your address, please contact our customer service team for further assistance.

Remember, you can also track your packages and view your order history in the "Account" or "Settings" tab. If you have any other questions or concerns, feel free to reach out to us.


## Obtaining Responses from OpenAI

Here, we demonstrate how to generate responses to our customer service requests using the OpenAI API. We wil be using the `gpt-3.5-turbo` model in this notebook.

Firstly, we need to enter our OpenAI API key.

In [10]:
openai.api_key = "<insert API key>"

Next, we define a function to format our prompts into the appropriate format to pass to the OpenAI API.

In [12]:
def format_openai_prompt(text):
    return [
        {"role": "system", "content": "You are a customer service agent, provide a response with next steps to the following question in a few sentences."},
        {"role": "user", "content": text}
    ]

Then, we pass the requests to the LLM using the formatted prompt and save its outputs.

In [ ]:
openai_outputs = np.zeros(len(requests)).astype(object)

for i, text in tqdm(enumerate(requests)):
    output = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=format_openai_prompt(text),
        temperature=0.8,  # can adjust temp 
    ) 
    
    openai_outputs[i] = output.choices[0].message.content

Finally, let's view a sample response from GPT-3.5:

In [14]:
print(openai_outputs[0])

I apologize for any inconvenience you may be facing with updating your delivery address. In order to assist you further, please provide me with your order number and the new delivery address. Once I have this information, I will be able to initiate the process of updating the delivery address for you. Thank you for your patience.
